## School Education Satistics

### Import Necessary Packages


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import col, avg
session = get_active_session()

### Load CSVs into DataFrames

In [ ]:
dor_df = session.table("DOR_DATA")
ger_df = session.table("GER_DATA")
btoilet_df = session.table("SCHOOL_WITH_BTOILET")
gtoilet_df = session.table("SCHOOL_WITH_GTOILET")
water_df = session.table("SCHOOL_WITH_WATER")

### Describe the data (summary statistics)

In [ ]:
dor_df.describe().show()
ger_df.describe().show()
btoilet_df.describe().show()
gtoilet_df.describe().show()
water_df.describe().show()

### Loop through each dataset and calculate missing values

In [ ]:
# Define the datasets
datasets = {
    "DOR": dor_df,
    "GER_1": ger_df,
    "school_with_btoilet": btoilet_df,
    "school_with_gtoilet": gtoilet_df,
    "school_with_water": water_df
}

for dataset_name, df in datasets.items():
    print(f"Missing values for {dataset_name} dataset:")
    
    # Calculate missing values for each column
    missing_values = df.select(
        [sum(col(c).isNull().cast("int")).alias(c + "_missing") for c in df.columns]
    )
    
    # Show missing values
    missing_values.show()

### Gross Enrolment Ratio from 2013-14 to 2015-16

In [ ]:
# Load data from Snowflake table
ger_df = session.table("GER_DATA").to_pandas()
# Display the first few rows of the DataFrame
print(ger_df.head())

##### Average Gross Enrollment Ratio (GER) by Education Level (2013-2014 to 2015-2016

In [ ]:
# Load data from Snowflake table
ger_df = session.table("GER_DATA").to_pandas()
# Data Cleaning: Convert columns to appropriate data types if needed
ger_df['YEAR'] = ger_df['YEAR'].astype(str)
# Calculate average GER by education level
avg_ger = ger_df.groupby('YEAR')[['PRIMARY_TOTAL', 'UPPER_PRIMARY_TOTAL', 'SECONDARY_TOTAL', 'HIGHER_SECONDARY_TOTAL']].mean()
avg_ger.plot(kind='bar')
plt.title('Average Gross Enrollment Ratio (GER) by Education Level (2013-2014 to 2015-2016)')
plt.xlabel('Year')
plt.ylabel('Average GER')
plt.xticks(rotation=45)
plt.legend(title='Education Level', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(axis='y')
plt.tight_layout()
plt.show()





##### Average Gross Enrollment Ratio by State/UT (2013-2014)

In [ ]:


# Calculate the total GER across all education levels for each state/UT and year
ger_df['TOTAL_GER'] = ger_df[['PRIMARY_TOTAL', 'UPPER_PRIMARY_TOTAL', 'SECONDARY_TOTAL', 'HIGHER_SECONDARY_TOTAL']].sum(axis=1)

# Identify the mean of total GER across all states/UTs and years
mean_ger = ger_df['TOTAL_GER'].mean()

# Count states/UTs with total GER greater than the mean
greater_than_mean_count = ger_df[ger_df['TOTAL_GER'] > mean_ger].shape[0]

# Count states/UTs with total GER less than or equal to the mean
less_than_equal_mean_count = ger_df[ger_df['TOTAL_GER'] <= mean_ger].shape[0]

# Display the counts
print("Count of States/UTs with Total GER Greater than Mean:", greater_than_mean_count)
print("Count of States/UTs with Total GER Less than or Equal to Mean:", less_than_equal_mean_count)

# Identify states/UTs with the lowest and highest GER
low_ger_state = ger_df.loc[ger_df['TOTAL_GER'].idxmin()]
high_ger_state = ger_df.loc[ger_df['TOTAL_GER'].idxmax()]

# Plotting the GER data for visualization
sns.barplot(x='STATE_UT', y='TOTAL_GER', data=ger_df, errorbar=None, color='lightblue')

# Overlay points for the states/UTs with lowest and highest GER
plt.scatter(low_ger_state['STATE_UT'], low_ger_state['TOTAL_GER'], color='red', s=100, label='Lowest GER')
plt.scatter(high_ger_state['STATE_UT'], high_ger_state['TOTAL_GER'], color='green', s=100, label='Highest GER')

# Adding titles and labels
plt.title('Total GER by State/UT (2013-2016)', fontsize=16)
plt.xlabel('State/UT', fontsize=14)
plt.ylabel('Total GER', fontsize=14)

# Adjust x-axis labels for better visibility
plt.xticks(rotation=90, ha='right')
plt.axhline(mean_ger, color='orange', linestyle='--', label='mean_ger')

# Adding legend
plt.legend()
plt.tight_layout()
plt.show()


### Schools with Boys Toilet from 2013-14 to 2015-16
##### Total Boys Toilets by State/UT (2013-2016)

In [ ]:
# Load data from Snowflake table
btoilet_df = session.table("SCHOOL_WITH_BTOILET").to_pandas()

# Ensure all column names are in uppercase
btoilet_df.columns = btoilet_df.columns.str.upper()

# Calculate the total count of boys' toilets across all years
btoilet_df['TOTAL_BOYS_TOILETS'] = btoilet_df[['PRIMARY_ONLY', 'PRIMARY_WITH_U_PRIMARY',
    'PRIMARY_WITH_U_PRIMARY_SEC_HRSEC', 'U_PRIMARY_ONLY', 'U_PRIMARY_WITH_SEC_HRSEC',
    'PRIMARY_WITH_U_PRIMARY_SEC', 'U_PRIMARY_WITH_SEC', 'SEC_ONLY', 
    'SEC_WITH_HRSEC', 'HRSEC_ONLY']].sum(axis=1)

# Identify the mean of total boys' toilets
mean_boys_toilets = btoilet_df['TOTAL_BOYS_TOILETS'].mean()

# Count states with total boys toilets greater than mean
greater_than_mean_count = btoilet_df[btoilet_df['TOTAL_BOYS_TOILETS'] > mean_boys_toilets].shape[0]

# Count states with total boys toilets less than or equal to mean
less_than_equal_mean_count = btoilet_df[btoilet_df['TOTAL_BOYS_TOILETS'] <= mean_boys_toilets].shape[0]

# Display counts
print("Count of States with Total Boys Toilets Greater than Mean:", greater_than_mean_count)
print("Count of States with Total Boys Toilets Less than or Equal to Mean:", less_than_equal_mean_count)

# Identify states with low and high boys' toilet availability
low_btoilet_state = btoilet_df.loc[btoilet_df['TOTAL_BOYS_TOILETS'].idxmin()]
high_btoilet_state = btoilet_df.loc[btoilet_df['TOTAL_BOYS_TOILETS'].idxmax()]

# Bar plot for total boys' toilets across all states without specifying a palette
sns.barplot(x='STATE_UT', y='TOTAL_BOYS_TOILETS', data=btoilet_df, errorbar=None, color='skyblue')

# Overlay points for the states with lowest and highest boys' toilet availability
plt.scatter(low_btoilet_state['STATE_UT'], low_btoilet_state['TOTAL_BOYS_TOILETS'], color='red', s=100, label='Lowest Boys Toilets')
plt.scatter(high_btoilet_state['STATE_UT'], high_btoilet_state['TOTAL_BOYS_TOILETS'], color='green', s=100, label='Highest Boys Toilets')

# Adding titles and labels
plt.title('', fontsize=16)
plt.xlabel('State/UT', fontsize=14)
plt.ylabel('Total Boys Toilets', fontsize=14)

# Adjust x-axis labels for better visibility
plt.xticks(rotation=90, ha='right')
plt.axhline(mean_boys_toilets, color='orange', linestyle='--', label='Mean Boys Toilets')

# Adding legend
plt.legend()
plt.tight_layout()
plt.show()

### Schools with Girls Toilet from 2013-14 to 2015-16
##### Total Girls Toilets by State/UT (2013-2016)

In [ ]:
# Load data from Snowflake table
gtoilet_df = session.table("SCHOOL_WITH_GTOILET").to_pandas()
# Ensure all column names are in uppercase
gtoilet_df.columns = gtoilet_df.columns.str.upper()

# Calculate the total count of girls' toilets across all years
gtoilet_df['TOTAL_GIRLS_TOILETS'] = gtoilet_df[['PRIMARY_ONLY', 'PRIMARY_WITH_U_PRIMARY',
    'PRIMARY_WITH_U_PRIMARY_SEC_HRSEC', 'U_PRIMARY_ONLY', 'U_PRIMARY_WITH_SEC_HRSEC',
    'PRIMARY_WITH_U_PRIMARY_SEC', 'U_PRIMARY_WITH_SEC', 'SEC_ONLY', 
    'SEC_WITH_HRSEC', 'HRSEC_ONLY', 'ALL_SCHOOLS']].sum(axis=1)

# Identify the mean of total girls' toilets
mean_girls_toilets = gtoilet_df['TOTAL_GIRLS_TOILETS'].mean()

# Count states with total girls' toilets greater than mean
greater_than_mean_count = gtoilet_df[gtoilet_df['TOTAL_GIRLS_TOILETS'] > mean_girls_toilets].shape[0]

# Count states with total girls' toilets less than or equal to mean
less_than_equal_mean_count = gtoilet_df[gtoilet_df['TOTAL_GIRLS_TOILETS'] <= mean_girls_toilets].shape[0]

# Display counts
print("Count of States with Total Girls Toilets Greater than Mean:", greater_than_mean_count)
print("Count of States with Total Girls Toilets Less than or Equal to Mean:", less_than_equal_mean_count)

# Identify states with low and high girls' toilet availability
low_gtoilet_state = gtoilet_df.loc[gtoilet_df['TOTAL_GIRLS_TOILETS'].idxmin()]
high_gtoilet_state = gtoilet_df.loc[gtoilet_df['TOTAL_GIRLS_TOILETS'].idxmax()]

# Bar plot for total girls' toilets across all states

sns.barplot(x='STATE_UT', y='TOTAL_GIRLS_TOILETS', data=gtoilet_df, errorbar=None, color='lightpink')

# Overlay points for the states with lowest and highest girls' toilet availability
plt.scatter(low_gtoilet_state['STATE_UT'], low_gtoilet_state['TOTAL_GIRLS_TOILETS'], color='red', s=100, label='Lowest Girls Toilets')
plt.scatter(high_gtoilet_state['STATE_UT'], high_gtoilet_state['TOTAL_GIRLS_TOILETS'], color='green', s=100, label='Highest Girls Toilets')

# Adding titles and labels
plt.title('Total Girls Toilets by State/UT (2013-2016)', fontsize=16)
plt.xlabel('State/UT', fontsize=14)
plt.ylabel('Total Girls Toilets', fontsize=14)

# Adjust x-axis labels for better visibility
plt.xticks(rotation=90, ha='right')
plt.axhline(mean_girls_toilets, color='orange', linestyle='--', label='Mean Girls Toilets')

# Adding legend
plt.legend()
plt.tight_layout()
plt.show()


### Percentage of Schools with Drinking Water Facility from 2013-14 to 2015-16
##### Total Water Availability by State/UT (2013-2016)

In [ ]:
# Load data from Snowflake table
water_df = session.table("SCHOOL_WITH_WATER").to_pandas()
# Ensure all column names are in uppercase
water_df.columns = water_df.columns.str.upper()
# Calculate the total count of water availability across all years
water_df['TOTAL_WATER_AVAILABILITY'] = water_df[['PRIMARY_ONLY', 'PRIMARY_WITH_U_PRIMARY',
    'PRIMARY_WITH_U_PRIMARY_SEC_HRSEC', 'U_PRIMARY_ONLY', 'U_PRIMARY_WITH_SEC_HRSEC',
    'PRIMARY_WITH_U_PRIMARY_SEC', 'U_PRIMARY_WITH_SEC', 'SEC_ONLY', 
    'SEC_WITH_HRSEC', 'HRSEC_ONLY', 'ALL_SCHOOLS']].sum(axis=1)

# Identify the mean of total water availability
mean_water_availability = water_df['TOTAL_WATER_AVAILABILITY'].mean()

# Count states with total water availability greater than mean
greater_than_mean_count = water_df[water_df['TOTAL_WATER_AVAILABILITY'] > mean_water_availability].shape[0]

# Count states with total water availability less than or equal to mean
less_than_equal_mean_count = water_df[water_df['TOTAL_WATER_AVAILABILITY'] <= mean_water_availability].shape[0]

# Display counts
print("Count of States with Total Water Availability Greater than Mean:", greater_than_mean_count)
print("Count of States with Total Water Availability Less than or Equal to Mean:", less_than_equal_mean_count)

# Identify states with low and high water availability
low_water_state = water_df.loc[water_df['TOTAL_WATER_AVAILABILITY'].idxmin()]
high_water_state = water_df.loc[water_df['TOTAL_WATER_AVAILABILITY'].idxmax()]

# Bar plot for total water availability across all states
sns.barplot(x='STATE_UT', y='TOTAL_WATER_AVAILABILITY', data=water_df, errorbar=None, color='lightblue')

# Overlay points for the states with lowest and highest water availability
plt.scatter(low_water_state['STATE_UT'], low_water_state['TOTAL_WATER_AVAILABILITY'], color='red', s=100, label='Lowest Water Availability')
plt.scatter(high_water_state['STATE_UT'], high_water_state['TOTAL_WATER_AVAILABILITY'], color='green', s=100, label='Highest Water Availability')

# Adding titles and labels
plt.title('Total Water Availability by State/UT (2013-2016)', fontsize=16)
plt.xlabel('State/UT', fontsize=14)
plt.ylabel('Total Water Availability', fontsize=14)

# Adjust x-axis labels for better visibility
plt.xticks(rotation=90, ha='right')
plt.axhline(mean_water_availability, color='orange', linestyle='--', label='Mean Water Availability')

# Adding legend
plt.legend()
plt.tight_layout()
plt.show()



### Drop-out Rate from 2012-13 to 2014-15
##### Primary Drop-out Rates by State/UT (2012-2015)

In [ ]:
# Load data from Snowflake table
dor_df = session.table("DOR_DATA").to_pandas()
# Ensure all column names are in uppercase
dor_df.columns = dor_df.columns.str.upper()

# Drop any rows with NR (Not Reported) or incorrect values, if needed
dor_df = dor_df.replace('NR', pd.NA)
dor_df = dor_df.dropna()

# Calculate the total drop-out rates across different education levels for each state and year
dor_df['PRIMARY_TOTAL_DROP_OUT'] = dor_df[['PRIMARY_BOYS', 'PRIMARY_GIRLS', 'PRIMARY_TOTAL']].mean(axis=1)
dor_df['UPPER_PRIMARY_TOTAL_DROP_OUT'] = dor_df[['UPPER_PRIMARY_BOYS', 'UPPER_PRIMARY_GIRLS', 'UPPER_PRIMARY_TOTAL']].mean(axis=1)
dor_df['SECONDARY_TOTAL_DROP_OUT'] = dor_df[['SECONDARY_BOYS', 'SECONDARY_GIRLS', 'SECONDARY_TOTAL']].mean(axis=1)
dor_df['HR_SECONDARY_TOTAL_DROP_OUT'] = dor_df[['HRSECONDARY_BOYS', 'HRSECONDARY_GIRLS', 'HRSECONDARY_TOTAL']].mean(axis=1)

# Calculate mean drop-out rates across all states
mean_primary_drop_out = dor_df['PRIMARY_TOTAL_DROP_OUT'].mean()
mean_upper_primary_drop_out = dor_df['UPPER_PRIMARY_TOTAL_DROP_OUT'].mean()
mean_secondary_drop_out = dor_df['SECONDARY_TOTAL_DROP_OUT'].mean()
mean_hr_secondary_drop_out = dor_df['HR_SECONDARY_TOTAL_DROP_OUT'].mean()

# Count states with drop-out rates greater than the mean for each education level
primary_above_mean_count = dor_df[dor_df['PRIMARY_TOTAL_DROP_OUT'] > mean_primary_drop_out].shape[0]
upper_primary_above_mean_count = dor_df[dor_df['UPPER_PRIMARY_TOTAL_DROP_OUT'] > mean_upper_primary_drop_out].shape[0]
secondary_above_mean_count = dor_df[dor_df['SECONDARY_TOTAL_DROP_OUT'] > mean_secondary_drop_out].shape[0]
hr_secondary_above_mean_count = dor_df[dor_df['HR_SECONDARY_TOTAL_DROP_OUT'] > mean_hr_secondary_drop_out].shape[0]

# Count states with drop-out rates less than or equal to the mean
primary_below_mean_count = dor_df[dor_df['PRIMARY_TOTAL_DROP_OUT'] <= mean_primary_drop_out].shape[0]
upper_primary_below_mean_count = dor_df[dor_df['UPPER_PRIMARY_TOTAL_DROP_OUT'] <= mean_upper_primary_drop_out].shape[0]
secondary_below_mean_count = dor_df[dor_df['SECONDARY_TOTAL_DROP_OUT'] <= mean_secondary_drop_out].shape[0]
hr_secondary_below_mean_count = dor_df[dor_df['HR_SECONDARY_TOTAL_DROP_OUT'] <= mean_hr_secondary_drop_out].shape[0]

# Print counts for each education level
print("Primary - States with Drop-out Rate Above Mean:", primary_above_mean_count)
print("Primary - States with Drop-out Rate Below or Equal to Mean:", primary_below_mean_count)
print("Upper Primary - States with Drop-out Rate Above Mean:", upper_primary_above_mean_count)
print("Upper Primary - States with Drop-out Rate Below or Equal to Mean:", upper_primary_below_mean_count)
print("Secondary - States with Drop-out Rate Above Mean:", secondary_above_mean_count)
print("Secondary - States with Drop-out Rate Below or Equal to Mean:", secondary_below_mean_count)
print("Higher Secondary - States with Drop-out Rate Above Mean:", hr_secondary_above_mean_count)
print("Higher Secondary - States with Drop-out Rate Below or Equal to Mean:", hr_secondary_below_mean_count)

# Identify states with lowest and highest drop-out rates for each level
low_primary_state = dor_df.loc[dor_df['PRIMARY_TOTAL_DROP_OUT'].idxmin()]
high_primary_state = dor_df.loc[dor_df['PRIMARY_TOTAL_DROP_OUT'].idxmax()]
low_secondary_state = dor_df.loc[dor_df['SECONDARY_TOTAL_DROP_OUT'].idxmin()]
high_secondary_state = dor_df.loc[dor_df['SECONDARY_TOTAL_DROP_OUT'].idxmax()]

# Bar plot for total drop-out rates across all states for Primary education level
sns.barplot(x='STATE_UT', y='PRIMARY_TOTAL_DROP_OUT', data=dor_df, errorbar=None, color='lightblue')

# Overlay points for the states with lowest and highest primary drop-out rates
plt.scatter(low_primary_state['STATE_UT'], low_primary_state['PRIMARY_TOTAL_DROP_OUT'], color='red', s=100, label='Lowest Primary Drop-out')
plt.scatter(high_primary_state['STATE_UT'], high_primary_state['PRIMARY_TOTAL_DROP_OUT'], color='green', s=100, label='Highest Primary Drop-out')

# Adding titles and labels
plt.title('Primary Drop-out Rates by State/UT (2012-2015)', fontsize=16)
plt.xlabel('State/UT', fontsize=14)
plt.ylabel('Primary Drop-out Rate', fontsize=14)

# Adjust x-axis labels for better visibility
plt.xticks(rotation=90, ha='right')
plt.axhline(mean_primary_drop_out, color='orange', linestyle='--', label='Mean Primary Drop-out Rate')

# Adding legend
plt.legend()
plt.tight_layout()
plt.show()


### Conclusion:
While Tamil Nadu excels in enrollment and school facilities, Nagaland lags in both infrastructure and educational outcomes. Water availability shows significant variation, with Delhi performing well but Nagaland and Mizoram struggling. Drop-out rates remain a challenge, particularly in higher grades.

This gives a holistic view of education access and infrastructure, highlighting regional disparities.